In [ ]:
import sys  # Interação com o sistema e caminhos do Python
from pathlib import Path  # Manipulação de caminhos de diretórios
import yfinance as yf  # Extração de dados financeiros do Yahoo Finance
import pandas as pd  # Manipulação e análise de dados (DataFrames)
from datetime import datetime, timedelta  # Manipulação de datas e prazos

# Define o caminho raiz do projeto (diretório pai)
root_path = str(Path.cwd().parent)

# Adiciona o caminho raiz ao sistema para permitir importações locais
if root_path not in sys.path:
    sys.path.append(root_path)

# Importa função de tratamento de nulos do utils.py (Ignora erro E402)
from utils import preencher_nulos  # noqa: E402

In [ ]:
# Define o ativo alvo (Itaú) e o período de coleta dos dados
ativo = "ITUB3.SA"
data_inicial = "2023-01-01"

# Define a data final como D-2 para evitar dados incompletos do pregão
data_final = (datetime.now() - timedelta(days=2)).strftime("%Y-%m-%d")

# Realiza o download dos dados históricos via API do Yahoo Finance
tabela_cotacao = yf.download(
    ativo,
    start=data_inicial,
    end=data_final,
    progress=False,
    multi_level_index=False,
)

# Calcula a variação nominal (R$) entre o fechamento atual e o anterior
tabela_cotacao["Variacao"] = tabela_cotacao["Close"].diff()

# Calcula a variação percentual (%) diária para a rede multivariada
tabela_cotacao["Var%"] = tabela_cotacao["Close"].pct_change() * 100

# Garante que o índice seja do tipo datetime para análise temporal
tabela_cotacao.index = pd.to_datetime(tabela_cotacao.index)

# Remove o nome das colunas para limpar a estrutura do DataFrame
tabela_cotacao.columns.name = None

# Identifica colunas que geram nulos (NaN) devido ao cálculo de diferença
col_nulo = ["Variacao", "Var%"]

# Substitui os valores nulos iniciais por 0 para evitar erros no modelo
tabela_cotacao = preencher_nulos(tabela_cotacao, col_nulo, 0)

# Exibe as primeiras linhas para validação do pré-processamento
tabela_cotacao.head(10)

,Close,High,Low,Open,Volume,Variacao,Var%
Date,,,,,,,
2023-01-02,14.359387,14.486581,14.178641,14.479886,544066,0.000000,0.000000
2023-01-03,14.084915,14.392856,13.971110,14.359383,899262,-0.274472,-1.911448
2023-01-04,14.098303,14.312522,14.031360,14.125081,908666,0.013388,0.095050
2023-01-05,14.513355,14.586993,14.111695,14.171945,1218768,0.415052,2.943988
2023-01-06,14.700802,14.767745,14.479887,14.546832,1122123,0.187447,1.291546
2023-01-09,14.740964,14.841379,14.566911,14.653938,1122916,0.040162,0.273197
2023-01-10,14.975270,15.022131,14.526748,14.687412,698267,0.234306,1.589491
2023-01-11,15.350149,15.397011,14.881545,14.961878,942316,0.374879,2.503320
2023-01-12,15.323372,15.450566,14.968572,15.283207,1505983,-0.026777,-0.174443


In [ ]:
# Define o caminho do diretório pai para organização das pastas
caminho_raiz = Path.cwd().parent

# Cria o caminho para a pasta de arquivos Parquet
pasta_parquet = caminho_raiz / "arquivos_parquet"

# Cria a pasta fisicamente se ela não existir (evita erros de diretório)
pasta_parquet.mkdir(parents=True, exist_ok=True)

# Define o nome e o caminho completo do arquivo de saída
caminho_arquivo = pasta_parquet / "dados_base.parquet"

# Salva o DataFrame no formato Parquet (otimizado para ML)
tabela_cotacao.to_parquet(caminho_arquivo)

# Exibe no console a confirmação de que o arquivo foi gerado com sucesso
print(f"Arquivo salvo em: {caminho_arquivo}")

Arquivo salvo em: c:\Users\Pc Gamer\OneDrive\FIAP\Machine Learnig Engineering\Fase - 04\fiap_tech_challenge_fase4\arquivos_parquet\dados_base.parquet
